# Quantify the Localisation of a (Basement Membrane) Protein Relative to the Acinus Exterior
- *Isobel Taylor-Hearn, 2024*
- Requires a 3D fluorescent image containg
    1. Protein of interest channel
    2. Nuclear channel and a membrane/cytoplasmic marker channel
        - These will be combined to approximate the acinar extent


In [ ]:
import os
import math
import pathlib
import warnings
from textwrap import wrap
from typing import Tuple, List

import numpy as np
import pandas as pd
import seaborn as sns
from scipy import ndimage as ndi
from scipy.ndimage import distance_transform_edt

import tifffile
from tifffile import imread

# --- Visualization ---
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import napari


from skimage import measure, util
from skimage.filters import threshold_otsu, gaussian
from skimage.measure import label, regionprops, regionprops_table
from skimage.morphology import remove_small_holes, remove_small_objects, ball, erosion

from skimage.segmentation import expand_labels
from skimage.transform import rescale

import contextlib
import joblib
from tqdm import tqdm
from joblib import Parallel, delayed

warnings.filterwarnings('ignore')

C:\Users\isobe\AppData\Local\Temp\ipykernel_25172\1375761233.py:17: DeprecationWarning: Please use `distance_transform_edt` from the `scipy.ndimage` namespace, the `scipy.ndimage.morphology` namespace is deprecated.
  from scipy.ndimage.morphology import distance_transform_edt


In [ ]:
@contextlib.contextmanager
def tqdm_joblib(tqdm_object):
    """
    Enables parallel jobs to run and display of a tqdm progress bar.

    Parameters:
    -----------
    tqdm_object : tqdm
        The tqdm progress bar instance to be updated.

    """
    class TqdmBatchCompletionCallback(joblib.parallel.BatchCompletionCallBack):
        def __call__(self, *args, **kwargs):
            tqdm_object.update(n=self.batch_size)
            return super().__call__(*args, **kwargs)

    old_batch_callback = joblib.parallel.BatchCompletionCallBack
    joblib.parallel.BatchCompletionCallBack = TqdmBatchCompletionCallback
    try:
        yield tqdm_object
    finally:
        joblib.parallel.BatchCompletionCallBack = old_batch_callback
        tqdm_object.close()

In [ ]:
def pixel_size(tif_path):
    """
    Extracts the pixel size (spacing) in microns from a TIFF image.

    This function reads TIFF metadata to determine the pixel spacing in the x, y, and z 
    dimensions, returning them as a list.

    Parameters:
    -----------
    tif_path : str
        Path to the TIFF file.

    Returns:
    --------
    original_spacing : list of float
        A list containing the pixel sizes in microns: [x_pixel_size_um, y_pixel_size_um, z_pixel_size_um].

    Notes:
    ------
    - The x and y resolutions are extracted from the TIFF XResolution and YResolution tags.
    - The z-spacing is inferred from the `IJMetadata` tag if available; otherwise, it falls back to 
      the `ImageDescription` tag.
    - Assumes the metadata is formatted in a way compatible with ImageJ or similar software.
    """
    with tifffile.TiffFile(tif_path) as tif:
        tif_tags = {}
        for tag in tif.pages[0].tags.values():
            name, value = tag.name, tag.value
            tif_tags[name] = value

        x_pixel_size_um = 1/((tif_tags["XResolution"])[0]/(tif_tags["XResolution"][1]))
        y_pixel_size_um = 1/((tif_tags["YResolution"])[0]/(tif_tags["YResolution"][1]))
        try:
            z_pixel_size_um = float(str(tif_tags["IJMetadata"]).split("nscales=")[1].split(",")[2].split("\\nunit")[0])
        except:
            z_pixel_size_um = (float(str(tif_tags["ImageDescription"]).split("spacing=")[1].split("loop")[0]))
       
    original_spacing = [x_pixel_size_um,y_pixel_size_um,z_pixel_size_um]
    return original_spacing

In [ ]:
def convert(img, target_type_min, target_type_max, target_type):
    """
    Converts an image to a specified data type while scaling its intensity values.

    This function rescales the intensity values of an image from its original range 
    to a new target range specified by `target_type_min` and `target_type_max`, and 
    then converts it to the desired data type.

    This step is required as deconvolved images are not always scaled 0->255! 

    Parameters:
    -----------
    img : numpy.ndarray
        The input image array to be converted.
    target_type_min : int or float
        The minimum value of the target intensity range.
    target_type_max : int or float
        The maximum value of the target intensity range.
    target_type : numpy.dtype
        The desired data type of the output image (e.g., np.uint8, np.float32).

    Returns:
    --------
    new_img : numpy.ndarray
        The rescaled image with values mapped to the new intensity range and converted 
        to the specified data type.

    Notes:
    ------
    - This function performs a linear transformation to scale pixel values.
    - It ensures that the output values are properly mapped between `target_type_min` and 
      `target_type_max`.
    """
    imin = img.min()
    imax = img.max()

    a = (target_type_max - target_type_min) / (imax - imin)
    b = target_type_max - a * imax
    new_img = (a * img + b).astype(target_type)
    return new_img

In [ ]:
def add_image_details(df, filename, flag):
    """
    Adds experimental details extracted from the filename to a dataframe.

    This function parses the filename to infer experimental details such as 
    well number, imaging day, mechanical stiffness condition, and treatment type.
    The extracted details are appended as new columns to the dataframe.

    Parameters:
    -----------
    df : pandas.DataFrame
        The dataframe to which image metadata will be added.
    filename : str
        The filename of the image, used to extract experimental details.
    flag : str
        A flag indicating any segmentation issues detected during processing.

    Returns:
    --------
    df : pandas.DataFrame
        The updated dataframe with the following added columns:
        - 'filename': The original filename.
        - 'flag': Segmentation flag indicating potential issues.
        - 'well': The well number (1 or 2) inferred from the filename.
        - 'day': The experimental time point (0, 1, 3, or 7 days).
        - 'condition': The mechanical stiffness condition ('soft', 'stiff', or 'blank').
        - 'treatment': The treatment applied ('blebbistatin', 'ROCKi', 'batimastat', 'ABT737', or 'none').
        - 'image_type': A combined descriptor of the condition and day (e.g., 'soft, d3').
    """
    df["filename"] = filename
    df["flag"] = flag
    if ("well1" in filename):
        df["well"] = 1
    else:
        df["well"] = 2
    # Day #        
    if "d0" in filename:
        df["day"] =  0
    elif "d1" in filename:
        df["day"] =  1
    elif "d3" in filename:
        df["day"] =  3
    else:
        df["day"] =  7
    # Stiffness
    if 'soft' in filename:
        df["condition"] =  'soft'
    elif 'stiff' in filename:
        df["condition"] =  'stiff'
    else:
        df["condition"] =  'blank'
    df['image_type'] = df["condition"].astype(str) + ", d" + df["day"].astype(str)

    return df

In [ ]:
def segment(path, DAPI_channel = 0, membrane_channel = 2, bm_channel=1, to_plot=True, colour="black"): 
    """
    Segment the primary acinus in a 3D multi-channel image and quantify basement membrane (BM) intensity as a function of 
    radial distance from the acinus center.

    Parameters:
    ----------
    path : str
        Path to the 3D multi-channel TIFF image file.
    DAPI_channel : int, default=0
        Channel index corresponding to the nuclear (DAPI) signal.
    membrane_channel : int, default=2
        Channel index corresponding to the membrane marker (e.g., CAAX).
    bm_channel : int, default=1
        Channel index corresponding to the basement membrane (e.g., laminin).
    to_plot : bool, default=True
        Whether to display and save diagnostic segmentation and quantification plots.
    colour : str, default="black"
        Colour to use for outlines and saved plot file naming.

    Returns:
    -------
    tuple or pd.DataFrame
        If `to_plot` is True:
            Returns a tuple of:
            - filtered_labelled_image : np.ndarray – Labelled binary mask of the segmented acinus
            - acinus_rescaled : np.ndarray – Rescaled intensity image used for segmentation
            - bm_rescaled : np.ndarray – Rescaled and masked BM channel within the segmented acinus
            - intensity_with_distance : pd.DataFrame – Average BM intensity as a function of normalised radial distance
        If `to_plot` is False:
            Returns only the `intensity_with_distance` DataFrame

    Notes:
    -----
    - The function combines DAPI and membrane channels to estimate the acinus region.
    - It uses smoothing, Otsu thresholding, and morphological operations to segment the largest acinus.
    - If the acinus is not spherical enough (based on inertia tensor), an erosion-based separation is applied.
    - BM intensity is then mapped relative to the acinus center using distance transform and normalised radius.
    - Optional plots show each step in the pipeline and a line plot of intensity vs. distance.
    - If image loading or segmentation fails, a flagged empty DataFrame is returned.
    """
    filename = os.path.basename(path).replace("_", "").lower()
    try:
        flag = "None"
        multi_channel_image = convert(imread(path), 0,255, np.uint8)
        acinus_image = (multi_channel_image[:,DAPI_channel,:,:] + multi_channel_image[:,membrane_channel,:,:]) # approximate the acinus as membrane + dapi
        bm_image = (multi_channel_image[:, bm_channel,:,:]) 

        original_spacing = pixel_size(path) #alter the z spacing so x,y,z all have the same pixel size
        scale_change = original_spacing[2] / original_spacing[0]
        acinus_rescaled = rescale(scale = (0.25*scale_change, 0.25, 0.25), image = acinus_image, anti_aliasing= False)
        bm_rescaled = rescale(scale = (0.25*scale_change, 0.25, 0.25), image = bm_image, anti_aliasing= False)
        clipped = acinus_rescaled.clip(min =np.quantile(acinus_rescaled, 0.1), max = np.quantile(acinus_rescaled, 0.85))
        acinus_smoothed = gaussian(clipped, sigma = 9)
        thresh = threshold_otsu(acinus_smoothed)
        binary = acinus_smoothed > thresh

        binary = remove_small_holes(binary, area_threshold = 100000)
        binary = remove_small_objects(binary, min_size = 10000)

        labelled_image = label(binary)
        table = regionprops_table(labelled_image, properties=('label', "area"),) #only keep largest acinus
        condition = (table['area'] >= (table["area"]).max())
        input_labels = table['label']
        output_labels = input_labels * condition
        filtered_labelled_image = util.map_array(labelled_image, input_labels, output_labels)

        table = regionprops_table(filtered_labelled_image, properties=('label', 'inertia_tensor_eigvals'),) #test sphericity of largest identified acinus
        condition = table['inertia_tensor_eigvals-2']/table['inertia_tensor_eigvals-0'] >= (0.55)
################################################################################################################
        if condition == False:
            #low sphericity could mean that 2 neighbouring acini have been imaged. Attempt to separate these with heavy erosion if so
            flag = "multiple_acini_split"
            acinus_smoothed = gaussian(clipped, sigma = 1)
            thresh = threshold_otsu(acinus_smoothed)
            binary = acinus_smoothed > thresh
            binary = remove_small_holes(binary, area_threshold = 100000)
            binary = remove_small_objects(binary, min_size = 10000)
            binary = erosion(binary, ball(8))
            labelled_image = label(binary)
            table = regionprops_table(labelled_image, properties=('label', "area"),)
            condition = (table['area'] >= (table["area"]).max())
            input_labels = table['label']
            output_labels = input_labels * condition
            filtered_labelled_image = util.map_array(labelled_image, input_labels, output_labels)
            filtered_labelled_image = expand_labels(filtered_labelled_image, distance=8)
            
    # ################################################################################################################
        bm_rescaled = bm_rescaled * (filtered_labelled_image) #clean bm image to focus on bm in/around identified acinus
        distance = (distance_transform_edt(filtered_labelled_image))
        r = np.cbrt((3* regionprops(filtered_labelled_image)[0].area)/(4*np.pi))
        distance = distance/r
        intensity_with_distance = pd.DataFrame(zip(np.ravel(distance), np.ravel(bm_rescaled)), columns=['distance/radius', 'bm_intensity'])    
        intensity_with_distance['rounded_distance'] = intensity_with_distance["distance/radius"].apply(lambda x: round(x, 2))
        intensity_with_distance = intensity_with_distance.groupby(['rounded_distance'], as_index=False)["bm_intensity"].mean()

        add_image_details(intensity_with_distance, filename, flag) #fill in df with experimental details extracted from filename
     ################################################################################################################   
        if to_plot == True:
            slice_to_display = int(filtered_labelled_image.shape[0]/2)
            fig, ax = plt.subplots(nrows = 1, ncols = 7, figsize=(20,4))
            ax[0].imshow(20*acinus_rescaled[slice_to_display,:,:] , cmap = "gray")
            ax[1].imshow(acinus_smoothed[slice_to_display,:,:], cmap="gray")
            ax[2].imshow(labelled_image[slice_to_display,:,:], cmap = "gray")
            ax[3].imshow(filtered_labelled_image[slice_to_display,:,:], cmap = "gray")
            ax[4].imshow(distance[slice_to_display,:,:], cmap = "gray")
            ax[5].imshow(bm_rescaled[slice_to_display,:,:], cmap = "gray")
            cols = ["Original", "Clipped",  "Identified Objects", "Largest Acinus", "Distance Map", "Clean BM", "Intensity v Dist"]
            for i in range(6):
                if i !=7:
                    ax[i].set_xticks([])
                    ax[i].set_yticks([])
                ax[i].set_title(cols[i]) 
            
            to_plot = intensity_with_distance.groupby(['rounded_distance'], as_index=False)["bm_intensity"].mean()
            to_plot["bm_intensity"] = to_plot["bm_intensity"]/ to_plot["bm_intensity"].max()
            sns.lineplot(data = to_plot, x="rounded_distance", y = "bm_intensity", ax = ax[-1], color="red")

            plt.savefig("Segmented{}.png".format(colour), bbox_inches="tight", transparent="True")

            return filtered_labelled_image, acinus_rescaled, bm_rescaled,  intensity_with_distance
        else:
            return intensity_with_distance
    except:
        intensity_with_distance = pd.DataFrame()
        intensity_with_distance["filename"] = filename
        intensity_with_distance["flag"]= "IMAGE FAILED"


## Single Run

In [ ]:
root = "E:\\PROTEIN_COLOCALISATION_IMAGES\\Deconvolved_Images\\Lam_Pax_DECONVOLVED\\final_tifs"
image_paths = list(pathlib.Path(root).glob("**/*.tif"))

In [ ]:
filtered_labelled_image, acinus_rescaled, laminin_rescaled,  intensity_with_distance = segment(image_paths[5], DAPI_channel = 0, membrane_channel = 2, bm_channel=1, to_plot=True)

In [11]:
viewer = napari.Viewer()
viewer.add_image(acinus_rescaled)
viewer.add_labels(filtered_labelled_image)
viewer.add_image(laminin_rescaled)

<Image layer 'laminin_rescaled' at 0x16168683520>

## Full Run with Parallelisation

In [12]:
with tqdm_joblib(tqdm(desc="Image Analysis", total=len(image_paths))) as progress_bar:
    intensity_with_distance = Parallel(n_jobs=3)(delayed(segment)(path, to_plot=False, bm_channel=1) for path in image_paths)

Image Analysis: 100%|██████████| 140/140 [31:26<00:00, 13.47s/it]


In [ ]:
total = pd.concat(intensity_with_distance).reset_index()
total.to_parquet('FINAL_laminin_290424.parquet')